In [139]:
import numpy as np

PATTERN_TO_DEBUG = 0

def toDec(hexstr,BIT):
    msb4bits = hexstr[0]
    n = int(msb4bits, 16)
    if n >= 8:
        p = -1*pow(2,BIT-1)
        addend = int(str(n-8) + hexstr[1:], 16)
        return str( p + addend)
    else:
        return str(int(hexstr, 16))

def int2hex(num,bit_width):
    if num<0:
        num = (1 << bit_width) + num
    hex_string = hex(num)[2:]
    return '{:0>{width}}'.format(hex_string,width = (bit_width+3)//4)

NEGATE = {'1': '0', '0': '1'}
def negate(value):
    return ''.join(NEGATE[x] for x in value)

def twocomplement(n, size_in_bits):
    # print(n)
    number = int(n)
    if number < 0:
        return negate(bin(abs(number) - 1)[2:]).rjust(size_in_bits, '1')
    else:
        return bin(number)[2:].rjust(size_in_bits, '0')

NEGATE = {'1': '0', '0': '1'}
def negate(value):
    return ''.join(NEGATE[x] for x in value)

def twocomplement(n, size_in_bits):
    # print(n)
    number = int(n)
    if number < 0:
        return negate(bin(abs(number) - 1)[2:]).rjust(size_in_bits, '1')
    else:
        return bin(number)[2:].rjust(size_in_bits, '0')


In [140]:
with open('input.txt', 'r') as file:
  file_in = file.readlines()

NUM = int(file_in[0])

# print(file_in)
for i in range(NUM):
  img_16 = []
  kernal_16 = []
  matrix_idx = []

  # MATRIX SIZE
  matrix_in_idx = int(file_in[1 + 50 * i + 0])

  if matrix_in_idx == 0:
     matrix_size = 8
  elif matrix_in_idx == 1:
     matrix_size = 16
  else:
     matrix_size = 32

  for j in range(16):
    img_in = np.array([int(toDec(val,8)) for val in file_in[1 + 50 * i + (1+j)].split()])
    img_16.append(np.reshape(img_in,(matrix_size,matrix_size)))

  for k in range(16):
    kernal_in = np.array([int(toDec(val,8)) for val in file_in[1 + 50 * i + (17+k)].split()])
    kernal_16.append(np.reshape(kernal_in,(5,5)))

  mode = int(file_in[1 + 50 * i + 33])

  for idx in range(16):
    matrix_idx.append(np.array([int(val) for val in file_in[1 + 50 * i + (34+idx)].split()]))

  if i == PATTERN_TO_DEBUG:
    break

In [141]:
img_16[0]

array([[2, 1, 1, 1, 1, 1, 1, 2],
       [1, 1, 1, 1, 2, 2, 2, 1],
       [1, 1, 1, 1, 2, 1, 1, 2],
       [1, 1, 1, 2, 1, 2, 2, 2],
       [2, 1, 1, 2, 1, 1, 1, 2],
       [1, 2, 2, 1, 1, 1, 2, 1],
       [1, 1, 1, 1, 2, 1, 1, 2],
       [2, 1, 2, 2, 1, 1, 2, 1]])

In [142]:
kernal_16[0]

array([[1, 1, 1, 1, 2],
       [2, 1, 2, 1, 1],
       [1, 1, 2, 1, 2],
       [2, 1, 2, 2, 1],
       [2, 2, 1, 2, 1]])

In [143]:
matrix_idx[0]

array([13,  8])

In [144]:
matrix_size

8

## Convolution

In [145]:
img = img_16[0]
kernal = kernal_16[0]

convoluted_results = np.zeros((matrix_size-4,matrix_size-4))
for i in range(matrix_size-4):
    for j in range(matrix_size-4):
        conv = 0
        for k in range(5):
            for l in range(5):
                conv += kernal[k,l] * img[i+k,j+l]

        convoluted_results[i,j] = conv

In [146]:
convoluted_results

array([[46., 43., 48., 55.],
       [47., 50., 50., 52.],
       [45., 47., 45., 53.],
       [49., 48., 51., 51.]])

## Maxpooling

In [147]:
convoluted_img_width = len(convoluted_results)

In [148]:
max_pooling_results = np.zeros((convoluted_img_width//2,convoluted_img_width//2))

for i in range(0,convoluted_img_width,2):
    for j in range(0,convoluted_img_width,2):
        max_pooling_results[i//2,j//2] = max(convoluted_results[i,j],convoluted_results[i+1,j],\
                                                convoluted_results[i+1,j+1],convoluted_results[i,j+1])

In [149]:
max_pooling_results

array([[50., 55.],
       [49., 53.]])

### Merged Convolution and max pooling

In [150]:
max_pooling_results = np.zeros((convoluted_img_width//2,convoluted_img_width//2))
for img_yptr in range(0,matrix_size-4,2):
    for img_xptr in range(0,matrix_size-4,2):
        temp_max = -999999
        # 4 times convolution to quickly get values needed for max pooling
        for mp_window_y in range(2):
            for mp_window_x in range(2):
                conv = 0
                idx_i = img_yptr+mp_window_y
                idx_j = img_xptr+mp_window_x
                for k_yptr in range(5):
                    for k_xptr in range(5):
                        conv += kernal[k,l] * img[idx_i+k_yptr,idx_j+k_xptr]

                if (mp_window_y == 0 and mp_window_x == 0) or conv > temp_max:
                    temp_max = conv

        max_pooling_results[img_yptr//2,img_xptr//2] = temp_max

In [151]:
max_pooling_results

array([[50., 55.],
       [49., 53.]])

### Transposed convolution

In [152]:
trans_convoluted_results = np.zeros((matrix_size+4,matrix_size+4))
for i in range(matrix_size):
    for j in range(matrix_size):
        for k in range(5):
            for l in range(5):
                conv = kernal[k,l] * img[i,j]
                trans_convoluted_results[i+k,j+l] += conv

In [153]:
trans_convoluted_results

array([[ 2.,  3.,  4.,  5.,  8.,  6.,  6.,  7.,  6.,  5.,  4.,  4.],
       [ 5.,  6., 10., 11., 15., 15., 16., 18., 15., 13.,  8.,  4.],
       [ 5.,  8., 14., 16., 25., 24., 26., 27., 24., 18., 11.,  9.],
       [ 8., 11., 19., 25., 33., 33., 37., 42., 33., 29., 19., 10.],
       [11., 17., 24., 35., 44., 45., 48., 54., 43., 33., 21., 13.],
       [10., 16., 26., 35., 45., 49., 50., 55., 40., 35., 23., 11.],
       [ 9., 17., 27., 36., 46., 47., 47., 52., 42., 32., 22., 13.],
       [11., 17., 28., 41., 49., 47., 53., 52., 39., 32., 24., 10.],
       [11., 17., 27., 36., 42., 38., 41., 38., 32., 24., 16.,  8.],
       [ 6., 12., 21., 25., 33., 30., 32., 32., 23., 19., 14.,  5.],
       [ 6.,  8., 14., 19., 22., 22., 22., 22., 17., 12.,  9.,  3.],
       [ 4.,  6.,  8., 13., 12., 11., 13., 11.,  7.,  6.,  4.,  1.]])

## Transpoed convolution with zero-padding

In [154]:
new_trans_convoluted_results = np.zeros((matrix_size+4,matrix_size+4))
lower_bound0 = matrix_size + 4
lower_bound1 = matrix_size + 5
lower_bound2 = matrix_size + 6
lower_bound3 = matrix_size + 7

for img_ytr in range(matrix_size+4):
    for img_xptr in range(matrix_size+4):
        conv = 0
        for k_yptr in range(5):
            for k_xptr in range(5):
                idx_i = img_ytr + k_yptr
                idx_j = img_xptr + k_xptr
                if idx_i == 0 or idx_i == 1 or idx_i == 2 or idx_i == 3 or idx_j == 0 or idx_j == 1 or idx_j == 2 or idx_j == 3:
                        conv += 0
                elif idx_i == lower_bound0 or idx_i == lower_bound1 or idx_i == lower_bound2 or idx_i == lower_bound3 \
                    or idx_j == lower_bound0 or idx_j == lower_bound1 or idx_j == lower_bound2 or idx_j == lower_bound3 :
                        conv += 0
                else:
                   conv += kernal[4-k_yptr,4-k_xptr] * img[idx_i-4,idx_j-4]

        new_trans_convoluted_results[img_ytr,img_xptr] = conv


In [155]:
new_trans_convoluted_results

array([[ 2.,  3.,  4.,  5.,  8.,  6.,  6.,  7.,  6.,  5.,  4.,  4.],
       [ 5.,  6., 10., 11., 15., 15., 16., 18., 15., 13.,  8.,  4.],
       [ 5.,  8., 14., 16., 25., 24., 26., 27., 24., 18., 11.,  9.],
       [ 8., 11., 19., 25., 33., 33., 37., 42., 33., 29., 19., 10.],
       [11., 17., 24., 35., 44., 45., 48., 54., 43., 33., 21., 13.],
       [10., 16., 26., 35., 45., 49., 50., 55., 40., 35., 23., 11.],
       [ 9., 17., 27., 36., 46., 47., 47., 52., 42., 32., 22., 13.],
       [11., 17., 28., 41., 49., 47., 53., 52., 39., 32., 24., 10.],
       [11., 17., 27., 36., 42., 38., 41., 38., 32., 24., 16.,  8.],
       [ 6., 12., 21., 25., 33., 30., 32., 32., 23., 19., 14.,  5.],
       [ 6.,  8., 14., 19., 22., 22., 22., 22., 17., 12.,  9.,  3.],
       [ 4.,  6.,  8., 13., 12., 11., 13., 11.,  7.,  6.,  4.,  1.]])

#### Convert results to LSB binary representation

In [156]:
twocomplement(-1,8)

'11111111'

In [157]:
vf = np.vectorize(twocomplement)
bits_result = vf(trans_convoluted_results,20)

In [158]:
bits_result[0][0]

'00000000000000000010'

In [159]:
result = bits_result[0][0]

In [160]:
result = result[::-1]

In [161]:
result

'01000000000000000000'